This notebook takes the raw data from the Data folder and cleans it up for analysis. It also creates a new dataset with the cleaned data.
It takes the data in wide format for each session and separately by treatment. It cleans the ego data and the stereotype data separately, homologates the variable names and then merges them together. The clean data is saved in the clean folder

In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import seaborn as sns
import random

# The Variables in the Raw Data
The data has the following structure:
Variables that start with the prefix 'participant.' are fixed throughout the experiment. 
Variables that start with the prefix 'Quizzes.' are the answers for part 1 of the experiment
Variables that start with the prefix 'Signals.' are the answers for part 2 of the experiment in the ego treatment
Variables that start with the prefix 'SignalsOther.' are the answers for part 2 of the experiment in the stereotype treatment
Variables that start with the prefix 'Questionnaire.' are the answers for gender and nationality at the end of the experiment

After the part-prefix, there is a number. The number indicates the round. Topics were randomized across rounds at the participant level. There were 11 rounds per topic in part 2. Hence the first guess of each topic is in rounds 1, 12, 23, 35, 46, 57. Whatever topic was not assigned in that round has empty values for that round.

**Participant Variables:**
- participant.code: The code of the participant
- participant.topic_score: The score of the participant in the topic (from 0 to 20)
- participant.topic_questions: A list of the questions in the order they saw them
- participant.gender: Their own gender identity (Male, Female, non-binary)
- participant.nationality: Their own nationality (US citizen, non-US citizen)

**Session level variables:**
- session.code: The code of the session
- session.seed: The random seed. It was fixed across all sessions
- session.w_topic: The omega drawn for each topic (also fixed across sessions because it uses the same seed)
- session.outcomes_topic: 3 matrixes with the outcomes for each possible choice of effort (low, mid, high)

**Variables from Part 1:**
- Quizzes.#.topic: The topic of the quiz from round # (only  one topic is answered in each round)
- Quizzes.#.guess: The guess of the participant for the topic in round #
- Quizzes.#.correct: Whether the guess was correct for the topic in round #
- Quizzez.#.topicn: The answer for the n-th question of the quiz in round #. there were 20 questions per quiz numbered from 0 to 19
- Quizzes.#.score: The score of the participant in round the topic drawn in round #

**Variables from Part 2 (Ego):**
In round 1 they answer questions asking what they think their score was and how certain they are of it
- Signals.#.topic: The topic of the signal from round # (only  one topic is answered in each round)
- Signals.1.topic_belief: The belief of the participant for the topic reported in the questionnaire right after the quizzes
- Signals.1.topic_certainty: The certainty of the participant for the topic reported in the questionnaire right after the quizzes (33, 50, 75, 100)
- Signals.#.player.effort: The effort of the participant for the topic in round #
- Signals.#.player.signal: The the number of successes the participant got in round # (out of 10)
- Signals.#.player.fails : The the number of failures the participant got in round # (out of 10)
- Signals.#.player.low_button: The number of times the clicked the to reveal the low matrix
- Signals.#.player.mid_button: The number of times the clicked the to reveal the mid matrix
- Signals.#.player.high_button: The number of times the clicked the to reveal the high matrix
- Signals.#.player.last_button: The last matrix that they chose to reveal before entering their effort choice
- Signals.subsession.round_number: The round number

**Variables from Part 2 (Stereotype):**
In round 1 they answer questions asking what they think their score was and how certain they are of it as well as for another participant. They only know the gender and the nationality of the other participant
- SignalsOther.#.topic: The topic of the signal from round # (only  one topic is answered in each round)
- SignalsOther.1.topic_belief_self: The belief of the participant for the topic reported in the questionnaire right after the quizzes
- Signals.1.topic_certainty_self: The certainty of the participant for the topic reported in the questionnaire right after the quizzes (33, 50, 75, 100)
- SignalsOther.1.topic_belief_other: The belief of the participant for the other participant in the topic
- SignalsOther.1.topic_certainty_other: The certainty of the participant for the other participant in the topic (33, 50, 75, 100)
- SignalsOther.#.player.effort: The effort of the participant for the topic in round #
- SignalsOther.#.player.signal: The the number of successes the participant got in round # (out of 10)
- SignalsOther.#.player.fails : The the number of failures the participant got in round # (out of 10)
- SignalsOther.#.player.low_button: The number of times the clicked the to reveal the low matrix
- SignalsOther.#.player.mid_button: The number of times the clicked the to reveal the mid matrix
- SignalsOther.#.player.high_button: The number of times the clicked the to reveal the high matrix
- SignalsOther.#.player.last_button: The last matrix that they chose to reveal before entering their effort choice
- SignalsOther.subsession.round_number: The round number
- SignalsOther.#.player.gender_other: The gender of the participant they were pairded with
- SignalsOther.#.player.nationality_other: The nationality of the participant they were pairded with

**Variables from the Questionnaire:**
- Questionnaire.1.player.gender: own gender
- Questionnaire.1.player.nationality: own nationality







### Clean the data for all the Ego sessions separately from all the stereotype sessions. I start with ego sessions.

In [3]:
# import the data in wide format for eah session
session1 = pd.read_csv("../Data/session1_ego.csv")
session2 = pd.read_csv("../Data/session2_ego.csv")
session3 = pd.read_csv("../Data/session3_ego.csv")
session4 = pd.read_csv("../Data/session4_ego.csv")

# concatenate the sessions
ego_wide = pd.concat([session1, session2, session3, session4], axis=0, sort=False)

ego_wide['treatment'] = 'ego'
ego_wide.reset_index(inplace=True)

# import the stereotype data in wide format
session5 = pd.read_csv("../Data/session5_stereo.csv")
session6 = pd.read_csv("../Data/session6_stereo.csv")
session7 = pd.read_csv("../Data/session7_stereo.csv")
session8 = pd.read_csv("../Data/session8_stereo.csv")
session9 = pd.read_csv("../Data/session9_stereo.csv")

stereo_wide = pd.concat([session5, session6, session7, session8, session9], axis=0, sort=False)
stereo_wide['treatment'] = 'stereotype'



/var/folders/fz/pp86wz7x4h514cs2lxtvtq3c0000gn/T/ipykernel_29368/3840415900.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ego_wide['treatment'] = 'ego'
/var/folders/fz/pp86wz7x4h514cs2lxtvtq3c0000gn/T/ipykernel_29368/3840415900.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ego_wide.reset_index(inplace=True)
/var/folders/fz/pp86wz7x4h514cs2lxtvtq3c0000gn/T/ipykernel_29368/3840415900.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

In [4]:
# from the data frame ego_wide, select all the columns that have names that start with 'Quizzes' and append the column 'participant.code
quiz_cols = [col for col in ego_wide.columns if 'Quizzes' in col]+['participant.code', 'treatment']


# from the data frame ego_wide, select all the columns that have names that start with 'participant.'
participant_cols = [col for col in ego_wide.columns if 'participant' in col]+['participant.code']

# from the data frame ego_wide, select all the columns that have names that start with 'session.'
session_cols = [col for col in ego_wide.columns if 'session' in col]+['participant.code']

# from the data frame ego_wide, select all the columns that have names that start with 'Signals.'
signal_cols = [col for col in ego_wide.columns if 'Signals' in col]+['participant.code']

# from the data frame ego_wide, select all the columns that have names that start with 'SignalsOther.'
signal_other_cols = [col for col in ego_wide.columns if 'SignalsOther' in col]+['participant.code']

# from the data frame ego_wide, select all the columns that have names that start with 'Questionnaire.'
questionnaire_cols = [col for col in ego_wide.columns if 'Questionnaire' in col]+['participant.code']


In [5]:
# split the data frame ego_wide into 6 data frames using the columns we just selected
ego_quiz = ego_wide[quiz_cols]
ego_participant = ego_wide[participant_cols]
ego_session = ego_wide[session_cols]
ego_signal = ego_wide[signal_cols]
ego_signal_other = ego_wide[signal_other_cols] # this has no information for the ego treatment
ego_questionnaire = ego_wide[questionnaire_cols]

## Quiz Scores
First I will clean the data for the first part of the experiment to get only the true scores.

In [6]:
# for ego_quiz rename all the columns to remove the prefix 'Quizzes'
ego_quiz.columns = [col.replace('Quizzes.','') for col in ego_quiz.columns]
# for ego_quiz rename all the columns to remove the prefix 'player'
ego_quiz.columns = [col.replace('player.','') for col in ego_quiz.columns]
# drop all the columns that have either 'group.' or 'subsession.' in the name
ego_quiz = ego_quiz[[col for col in ego_quiz.columns if 'group.' not in col and 'subsession.' not in col]]
# melt the data making the participant code the id variable
ego_quiz_long = pd.melt(ego_quiz, id_vars=['participant.code'])
# split the variable column into two columns, one for the round_number and one for the question
ego_quiz_long[['round_number','variable_name']] = ego_quiz_long['variable'].str.split('.', expand=True)
# drop the column variable with the long names
ego_quiz_long = ego_quiz_long.drop('variable', axis=1)
# reshape ego_quiz_long from long to wide format by making each of the values in variable_name a column
ego_quiz_wide = ego_quiz_long.pivot(index=['participant.code','round_number'], columns=['variable_name'], values='value')
# make a table that has only the score for each topic and the participant code
ego_scores = ego_quiz_wide[['topic', 'score']]
# reset the index so that participant_code is just another colum
ego_scores.reset_index(inplace=True)

### Participant level variables

In [7]:
# rename all the other variables withoutht the 'participant.' prefix
ego_participant.columns = [col.replace('participant.','') for col in ego_participant.columns]


## The variables from the main part of the experiment
This table has all the effort choices and signal realizations round by round. It also has the beliefs about their score and the buttons clicked for the type matixes

In [8]:
# from ego_signal drop all the columns that have the 'SignalsOther.' prefix
ego_signal = ego_signal[[col for col in ego_signal.columns if 'SignalsOther.' not in col]]

# remove the 'Signals.' prefix from all the column names and the 'participant.' prefix from the code column
ego_signal.columns = [col.replace('Signals.','') for col in ego_signal.columns]
ego_signal.columns = [col.replace('participant.','') for col in ego_signal.columns]
# replace the 'player.' in the names of the columns
ego_signal.columns = [col.replace('player.', '') for col in ego_signal.columns]

# drop the group and subsession level columns
ego_signal = ego_signal[[col for col in ego_signal.columns if 'group.' not in col and '.subsession' not in col]]

# melt the data set making code the id column. then split all the variable names
ego_signal_long = pd.melt(ego_signal, id_vars='code')

In [9]:
# split the variable column into two columns, one for the round_number and one for the question
ego_signal_long[['round_number','variable_name']] = ego_signal_long['variable'].str.split('.', expand=True)
# drop the column variable with the long names
ego_signal_long = ego_signal_long.drop('variable', axis=1)

# reshape ego_signal_long from long to wide format by making each of the values in variable_name a column
ego_signal_wide = ego_signal_long.pivot(index=['code','round_number'], columns=['variable_name'], values='value')
ego_signal_wide.reset_index(inplace=True)

In [10]:
beliefs = ego_signal_wide.loc[ego_signal_wide['science_belief']>=0, 
                    ['code','science_belief', 'us_belief', 'math_belief', 'verbal_belief', 'pop_belief', 'sports_belief',
                    'science_certainty', 'us_certainty', 'math_certainty', 'verbal_certainty', 'pop_certainty', 'sports_certainty']]

In [11]:
# the round_numbers range from 1 to 60. I want all of them to range from 1 to 11 by setting 11 to 1, 21 to 1, 12 to 2, etc.
ego_signal_wide['round_number'] = ego_signal_wide['round_number'].astype(int)

ego_signal_wide['round_number'].replace([12,23,34,45,56], 1, inplace=True)
ego_signal_wide['round_number'].replace([13,24,35,46,57], 2, inplace=True)
ego_signal_wide['round_number'].replace([14,25,36,47,58], 3, inplace=True)
ego_signal_wide['round_number'].replace([15,26,37,48,59], 4, inplace=True)
ego_signal_wide['round_number'].replace([16,27,38,49,60], 5, inplace=True)
ego_signal_wide['round_number'].replace([17,28,39,50,61], 6, inplace=True)
ego_signal_wide['round_number'].replace([18,29,40,51,62], 7, inplace=True)
ego_signal_wide['round_number'].replace([19,30,41,52,63], 8, inplace=True)
ego_signal_wide['round_number'].replace([20,31,42,53,64], 9, inplace=True)
ego_signal_wide['round_number'].replace([21,32,43,54,65], 10, inplace=True)
ego_signal_wide['round_number'].replace([22,33,44,55,66], 11, inplace=True)


### Split the signal stage by topic. 
For all participants, they got the same exogenous rate and the same order of signals for each topic.
I will then merge the topics that have the same exchange rate and the same order of signals.

there are nine combinations of type and rate that each participant could have had in each topic. Those 9 pairs are the ones I am interested in. 

In [12]:
ego_science = ego_signal_wide.loc[(ego_signal_wide['effort']>=0) & (ego_signal_wide['topic']=='Science and Technology'), 
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'science_score', 'topic', 'signal']]
# impute the values for science_belief and the science_certainty from the table beliefs
ego_science = ego_science.merge(beliefs[['code', 'science_belief', 'science_certainty']], on='code', how='left')

# add a column with the type according to the science score
ego_science.loc[ego_science['science_score']>15, 'type'] = '2'
ego_science.loc[ego_science['science_score']<=15, 'type'] = '1'
ego_science.loc[ego_science['science_score']<6, 'type'] = '0'

# make a data frame for math exactly as ego_science but with the math score and belief
ego_math = ego_signal_wide.loc[(ego_signal_wide['effort']>=0) & (ego_signal_wide['topic']=='Math'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'math_score', 'topic', 'signal']]
# impute the values for math_belief and the math_certainty from the table beliefs
ego_math = ego_math.merge(beliefs[['code', 'math_belief', 'math_certainty']], on='code', how='left')

# add a column with the type according to the science score
ego_math.loc[ego_math['math_score']>15, 'type'] = '2'
ego_math.loc[ego_math['math_score']<=15, 'type'] = '1'
ego_math.loc[ego_math['math_score']<6, 'type'] = '0'

# make a data frame for verbal exactly as ego_science but with the verbal score and belief
ego_verbal = ego_signal_wide.loc[(ego_signal_wide['effort']>=0) & (ego_signal_wide['topic']=='Verbal'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'verbal_score', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
ego_verbal = ego_verbal.merge(beliefs[['code', 'verbal_belief', 'verbal_certainty']], on='code', how='left')

# add a column with the type according to the science score
ego_verbal.loc[ego_verbal['verbal_score']>15, 'type'] = '2'
ego_verbal.loc[ego_verbal['verbal_score']<=15, 'type'] = '1'
ego_verbal.loc[ego_verbal['verbal_score']<6, 'type'] = '0'

# make a data frame for verbal exactly as ego_science but with the verbal score and belief
ego_pop = ego_signal_wide.loc[(ego_signal_wide['effort']>=0) & (ego_signal_wide['topic']=='Pop-Culture and Art'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'pop_score', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
ego_pop = ego_pop.merge(beliefs[['code', 'pop_belief', 'pop_certainty']], on='code', how='left')

# add a column with the type according to the science score
ego_pop.loc[ego_pop['pop_score']>15, 'type'] = '2'
ego_pop.loc[ego_pop['pop_score']<=15, 'type'] = '1'
ego_pop.loc[ego_pop['pop_score']<6, 'type'] = '0'

# make a data frame for verbal exactly as ego_science but with the verbal score and belief
ego_sports = ego_signal_wide.loc[(ego_signal_wide['effort']>=0) & (ego_signal_wide['topic']=='Sports and Video Games'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'sports_score', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
ego_sports = ego_sports.merge(beliefs[['code', 'sports_belief', 'sports_certainty']], on='code', how='left')

# add a column with the type according to the science score
ego_sports.loc[ego_sports['sports_score']>15, 'type'] = '2'
ego_sports.loc[ego_sports['sports_score']<=15, 'type'] = '1'
ego_sports.loc[ego_sports['sports_score']<6, 'type'] = '0'

# make a data frame for verbal exactly as ego_science but with the verbal score and belief
ego_us = ego_signal_wide.loc[(ego_signal_wide['effort']>=0) & (ego_signal_wide['topic']=='US Geography'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'us_score', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
ego_us = ego_us.merge(beliefs[['code', 'us_belief', 'us_certainty']], on='code', how='left')

# add a column with the type according to the science score
ego_us.loc[ego_us['us_score']>15, 'type'] = '2'
ego_us.loc[ego_us['us_score']<=15, 'type'] = '1'
ego_us.loc[ego_us['us_score']<6, 'type'] = '0'

## Get the rate values for each topic
The rate values are saved at the session level and are the same across all rounds because the seed was set at the beginning

In [13]:
# from the ego session data frame select only the columns that we will use
ego_session = ego_session[['participant.id_in_session', 
                            'session.code', 
                            'session.w_verbal', 
                            'session.w_math', 
                            'session.w_pop',
                            'session.w_science',
                            'session.w_sports',
                            'session.w_us']]


In [14]:
# in ego_session remove the prefixes 'participant.' and 'session.' from the column names
ego_session.columns = [col.replace('participant.','') for col in ego_session.columns]
ego_session.columns = [col.replace('session.','') for col in ego_session.columns]
# add a column to each of the topics tables with the corresponding value of w from the ego_session table
ego_science['rate']=ego_session['w_science'][0]
ego_math['rate']=ego_session['w_math'][0]
ego_verbal['rate']=ego_session['w_verbal'][0]
ego_pop['rate']=ego_session['w_pop'][0]
ego_sports['rate']=ego_session['w_sports'][0]
ego_us['rate']=ego_session['w_us'][0]

## name homogenization
Make all the toipic tables have the same column names so that they can be merged into one table

In [15]:
# from ego_science remove the prefix 'science_' from the column names that have it
ego_science.columns = [col.replace('science_','') for col in ego_science.columns]

# from ego_math remove the prefix 'math_' from the column names that have it
ego_math.columns = [col.replace('math_','') for col in ego_math.columns]

# from ego_verbal remove the prefix 'verbal_' from the column names that have it
ego_verbal.columns = [col.replace('verbal_','') for col in ego_verbal.columns]

# from ego_pop remove the prefix 'pop_' from the column names that have it
ego_pop.columns = [col.replace('pop_','') for col in ego_pop.columns]

# from ego_sports remove the prefix 'sports_' from the column names that have it
ego_sports.columns = [col.replace('sports_','') for col in ego_sports.columns]

# from ego_us remove the prefix 'us_' from the column names that have it
ego_us.columns = [col.replace('us_','') for col in ego_us.columns]

# stack all the topic tables into one larger data frame called ego_updates
ego_updates = pd.concat([ego_science, ego_math, ego_verbal, ego_pop, ego_sports, ego_us])
ego_updates.reset_index(inplace=True, drop=True)


## separate into 9 types:
low type, low rate

low type, medium rate

low type, high rate

medium type, low rate

medium type, medium rate

medium type, high rate

high type, low rate

high type, medium rate

high type, high rate

In [16]:
# add a colun that indicates the treatment
ego_updates['treatment'] = 'ego'

# turn the type column into an integer
ego_updates['type'] = ego_updates['type'].astype(int)

# add a column that indicates if the participant was overconfident
ego_updates['overconfident'] = 0
ego_updates.loc[ego_updates['type']<ego_updates['belief'], 'overconfident'] = 1

# add a column that indicates if the participant was underconfident
ego_updates['underconfident'] = 0
ego_updates.loc[ego_updates['type']>ego_updates['belief'], 'underconfident'] = 1

# add a column that indicates if the participant was correct
ego_updates['correct']=0
ego_updates.loc[ego_updates['type']==ego_updates['belief'], 'correct'] = 1

# create a table for each pair of theta and omega (type and rate) the subindexes are in that order
#low types
updates_ll = ego_updates.loc[(ego_updates['type']==0) & (ego_updates['rate']==0), :]
updates_lm = ego_updates.loc[(ego_updates['type']==0) & (ego_updates['rate']==1), :]
updates_lh = ego_updates.loc[(ego_updates['type']==0) & (ego_updates['rate']==2), :]

# mid types
updates_ml = ego_updates.loc[(ego_updates['type']==1) & (ego_updates['rate']==0), :]
updates_mm = ego_updates.loc[(ego_updates['type']==1) & (ego_updates['rate']==1), :]
updates_mh = ego_updates.loc[(ego_updates['type']==1) & (ego_updates['rate']==2), :]

# high types
updates_hl = ego_updates.loc[(ego_updates['type']==2) & (ego_updates['rate']==0), :]
updates_hm = ego_updates.loc[(ego_updates['type']==2) & (ego_updates['rate']==1), :]
updates_hh = ego_updates.loc[(ego_updates['type']==2) & (ego_updates['rate']==2), :]

In [17]:
ego_updates.loc[ego_updates['overconfident']==1, 'misspecification'] = 'over'
ego_updates.loc[ego_updates['underconfident']==1, 'misspecification'] = 'under'
ego_updates.loc[ego_updates['correct']==1, 'misspecification'] = 'correct'
ego_updates

,code,round_number,effort,fails,last_button,score,topic,signal,belief,certainty,type,rate,treatment,overconfident,underconfident,correct,misspecification
0,0m2xzxgv,1,1,9,2,9.0,Science and Technology,1,0,75,1,0,ego,0,1,0,under
1,0m2xzxgv,2,1,7,1,9.0,Science and Technology,3,0,75,1,0,ego,0,1,0,under
2,0m2xzxgv,3,2,10,2,9.0,Science and Technology,0,0,75,1,0,ego,0,1,0,under
3,0m2xzxgv,4,0,7,0,9.0,Science and Technology,3,0,75,1,0,ego,0,1,0,under
4,0m2xzxgv,5,0,8,1,9.0,Science and Technology,2,0,75,1,0,ego,0,1,0,under
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2965,zap4p51l,7,1,9,0,5.0,US Geography,1,0,75,0,1,ego,0,0,1,correct
2966,zap4p51l,8,0,9,0,5.0,US Geography,1,0,75,0,1,ego,0,0,1,correct
2967,zap4p51l,9,2,10,0,5.0,US Geography,0,0,75,0,1,ego,0,0,1,correct
2968,zap4p51l,10,1,5,0,5.0,US Geography,5,0,75,0,1,ego,0,0,1,correct


# The Stereotypes treatment
do the same for all tables and instead of using signals, use SignalsOther. Then merge the two tables together with the column that indicates the treatment

In [18]:
# from the data frame select all the columns that have names that start with 'Quizzes' and append the column 'participant.code
quiz_cols = [col for col in stereo_wide.columns if 'Quizzes' in col]+['participant.code', 'treatment']


# from the data frame select all the columns that have names that start with 'participant.'
participant_cols = [col for col in stereo_wide.columns if 'participant' in col]+['participant.code']

# from the data frame select all the columns that have names that start with 'session.'
session_cols = [col for col in stereo_wide.columns if 'session' in col]+['participant.code']

# from the data frame select all the columns that have names that start with 'Signals.'
signal_cols = [col for col in stereo_wide.columns if 'Signals' in col]+['participant.code']

# from the data frame select all the columns that have names that start with 'SignalsOther.'
signal_other_cols = [col for col in stereo_wide.columns if 'SignalsOther' in col]+['participant.code']

# from the data frame select all the columns that have names that start with 'Questionnaire.'
questionnaire_cols = [col for col in stereo_wide.columns if 'Questionnaire' in col]+['participant.code']

# split the data frame stereo_wide into 6 data frames using the columns we just selected
stereo_quiz = stereo_wide[quiz_cols]
stereo_participant = stereo_wide[participant_cols]
stereo_session = stereo_wide[session_cols]
stereo_signal = stereo_wide[signal_cols] # this has no information for the stereotype treatment
stereo_signal_other = stereo_wide[signal_other_cols] 
stereo_questionnaire = stereo_wide[questionnaire_cols]

# for stereo_quiz rename all the columns to remove the prefix 'Quizzes'
stereo_quiz.columns = [col.replace('Quizzes.','') for col in stereo_quiz.columns]
# for stereo_quiz rename all the columns to remove the prefix 'player'
stereo_quiz.columns = [col.replace('player.','') for col in stereo_quiz.columns]

# drop all the columns that have either 'group.' or 'subsession.' in the name
stereo_quiz = stereo_quiz[[col for col in stereo_quiz.columns if 'group.' not in col and 'subsession.' not in col]]
# melt the data making the participant code the id variable
stereo_quiz_long = pd.melt(stereo_quiz, id_vars=['participant.code'])
# split the variable column into two columns, one for the round_number and one for the question
stereo_quiz_long[['round_number','variable_name']] = stereo_quiz_long['variable'].str.split('.', expand=True)
# drop the column variable with the long names
stereo_quiz_long = stereo_quiz_long.drop('variable', axis=1)
# reshape stereo_quiz_long from long to wide format by making each of the values in variable_name a column
stereo_quiz_wide = stereo_quiz_long.pivot(index=['participant.code','round_number'], columns=['variable_name'], values='value')
# make a table that has only the score for each topic and the participant code
stereo_scores = stereo_quiz_wide[['topic', 'score']]
# reset the index so that participant_code is just another colum
stereo_scores.reset_index(inplace=True)

# Rename all the other variables withoutht the 'participant.' prefix
stereo_participant.columns = [col.replace('participant.','') for col in stereo_participant.columns]

### Main part of the experiment for the sterotype treatment

# remove the 'Signals.' prefix from all the column names and the 'participant.' prefix from the code column
stereo_signal_other.columns = [col.replace('SignalsOther.','') for col in stereo_signal_other.columns]
stereo_signal_other.columns = [col.replace('participant.','') for col in stereo_signal_other.columns]
# replace the 'player.' in the names of the columns
stereo_signal_other.columns = [col.replace('player.', '') for col in stereo_signal_other.columns]

# drop the group and subsession level columns
stereo_signal_other = stereo_signal_other[[col for col in stereo_signal_other.columns if 'group.' not in col and '.subsession' not in col]]

# melt the data set making code the id column. then split all the variable names
stereo_signal_other_long = pd.melt(stereo_signal_other, id_vars='code')

# split the variable column into two columns, one for the round_number and one for the question
stereo_signal_other_long[['round_number','variable_name']] = stereo_signal_other_long['variable'].str.split('.', expand=True)
# drop the column variable with the long names
stereo_signal_other_long = stereo_signal_other_long.drop('variable', axis=1)

# reshape stereo_signal_other_long from long to wide format by making each of the values in variable_name a column
stereo_signal_other_wide = stereo_signal_other_long.pivot(index=['code','round_number'], columns=['variable_name'], values='value')
stereo_signal_other_wide.reset_index(inplace=True)

beliefs_other = stereo_signal_other_wide.loc[stereo_signal_other_wide['science_belief_other']>=0, 
                    ['code','science_belief_other', 'us_belief_other', 'math_belief_other', 'verbal_belief_other', 
                    'pop_belief_other', 'sports_belief_other',
                    'science_certainty_other', 'us_certainty_other', 'math_certainty_other', 'verbal_certainty_other', 
                    'pop_certainty_other', 'sports_certainty_other']]

# the round_numbers range from 1 to 60. I want all of them to range from 1 to 10 by setting 11 to 1, 21 to 1, 12 to 2, etc.
stereo_signal_other_wide['round_number'] = stereo_signal_other_wide['round_number'].astype(int)

stereo_signal_other_wide['round_number'].replace([12,23,34,45,56], 1, inplace=True)
stereo_signal_other_wide['round_number'].replace([13,24,35,46,57], 2, inplace=True)
stereo_signal_other_wide['round_number'].replace([14,25,36,47,58], 3, inplace=True)
stereo_signal_other_wide['round_number'].replace([15,26,37,48,59], 4, inplace=True)
stereo_signal_other_wide['round_number'].replace([16,27,38,49,60], 5, inplace=True)
stereo_signal_other_wide['round_number'].replace([17,28,39,50,61], 6, inplace=True)
stereo_signal_other_wide['round_number'].replace([18,29,40,51,62], 7, inplace=True)
stereo_signal_other_wide['round_number'].replace([19,30,41,52,63], 8, inplace=True)
stereo_signal_other_wide['round_number'].replace([20,31,42,53,64], 9, inplace=True)
stereo_signal_other_wide['round_number'].replace([21,32,43,54,65], 10, inplace=True)
stereo_signal_other_wide['round_number'].replace([22,33,44,55,66], 11, inplace=True)

# split by topic

st_science = stereo_signal_other_wide.loc[(stereo_signal_other_wide['effort']>=0) & (stereo_signal_other_wide['topic']=='Science and Technology'), 
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'science_other', 'topic', 'signal']]
# impute the values for science_belief and the science_certainty from the table beliefs
st_science = st_science.merge(beliefs_other[['code', 'science_belief_other', 'science_certainty_other']], on='code', how='left')

# add a column with the type according to the science score
st_science.loc[st_science['science_other']>15, 'type'] = '2'
st_science.loc[st_science['science_other']<=15, 'type'] = '1'
st_science.loc[st_science['science_other']<6, 'type'] = '0'

# make a data frame for math exactly as st_science but with the math score and belief
st_math = stereo_signal_other_wide.loc[(stereo_signal_other_wide['effort']>=0) & (stereo_signal_other_wide['topic']=='Math'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'math_other', 'topic', 'signal']]
# impute the values for math_belief and the math_certainty from the table beliefs
st_math = st_math.merge(beliefs_other[['code', 'math_belief_other', 'math_certainty_other']], on='code', how='left')

# add a column with the type according to the science score
st_math.loc[st_math['math_other']>15, 'type'] = '2'
st_math.loc[st_math['math_other']<=15, 'type'] = '1'
st_math.loc[st_math['math_other']<6, 'type'] = '0'

# make a data frame for verbal exactly as st_science but with the verbal score and belief
st_verbal = stereo_signal_other_wide.loc[(stereo_signal_other_wide['effort']>=0) & (stereo_signal_other_wide['topic']=='Verbal'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'verbal_other', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
st_verbal = st_verbal.merge(beliefs_other[['code', 'verbal_belief_other', 'verbal_certainty_other']], on='code', how='left')

# add a column with the type according to the science score
st_verbal.loc[st_verbal['verbal_other']>15, 'type'] = '2'
st_verbal.loc[st_verbal['verbal_other']<=15, 'type'] = '1'
st_verbal.loc[st_verbal['verbal_other']<6, 'type'] = '0'

# make a data frame for verbal exactly as st_science but with the verbal score and belief
st_pop = stereo_signal_other_wide.loc[(stereo_signal_other_wide['effort']>=0) & (stereo_signal_other_wide['topic']=='Pop-Culture and Art'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'pop_other', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
st_pop = st_pop.merge(beliefs_other[['code', 'pop_belief_other', 'pop_certainty_other']], on='code', how='left')

# add a column with the type according to the science score
st_pop.loc[st_pop['pop_other']>15, 'type'] = '2'
st_pop.loc[st_pop['pop_other']<=15, 'type'] = '1'
st_pop.loc[st_pop['pop_other']<6, 'type'] = '0'

# make a data frame for verbal exactly as st_science but with the verbal score and belief
st_sports = stereo_signal_other_wide.loc[(stereo_signal_other_wide['effort']>=0) & (stereo_signal_other_wide['topic']=='Sports and Video Games'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'sports_other', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
st_sports = st_sports.merge(beliefs_other[['code', 'sports_belief_other', 'sports_certainty_other']], on='code', how='left')

# add a column with the type according to the science score
st_sports.loc[st_sports['sports_other']>15, 'type'] = '2'
st_sports.loc[st_sports['sports_other']<=15, 'type'] = '1'
st_sports.loc[st_sports['sports_other']<6, 'type'] = '0'

# make a data frame for verbal exactly as st_science but with the verbal score and belief
st_us = stereo_signal_other_wide.loc[(stereo_signal_other_wide['effort']>=0) & (stereo_signal_other_wide['topic']=='US Geography'),
                    ['code', 'round_number', 'effort', 'fails', 'last_button', 'us_other', 'topic', 'signal']]
# impute the values for verbal_belief and the verbal_certainty from the table beliefs
st_us = st_us.merge(beliefs_other[['code', 'us_belief_other', 'us_certainty_other']], on='code', how='left')

# add a column with the type according to the science score
st_us.loc[st_us['us_other']>15, 'type'] = '2'
st_us.loc[st_us['us_other']<=15, 'type'] = '1'
st_us.loc[st_us['us_other']<6, 'type'] = '0'

# from the ego session data frame select only the columns that we will use
stereo_session = stereo_session[['participant.id_in_session', 
                            'session.code', 
                            'session.w_verbal', 
                            'session.w_math', 
                            'session.w_pop',
                            'session.w_science',
                            'session.w_sports',
                            'session.w_us']]


In [19]:
# in stereo_session remove the prefixes 'participant.' and 'session.' from the column names
stereo_session.columns = [col.replace('participant.','') for col in stereo_session.columns]
stereo_session.columns = [col.replace('session.','') for col in stereo_session.columns]
# add a column to each of the topics tables with the corresponding value of w from the stereo_session table
st_science['rate']=stereo_session['w_science'].unique()[0]
st_math['rate']=stereo_session['w_math'].unique()[0]
st_verbal['rate']=stereo_session['w_verbal'].unique()[0]
st_pop['rate']=stereo_session['w_pop'].unique()[0]
st_sports['rate']=stereo_session['w_sports'].unique()[0]
st_us['rate']=stereo_session['w_us'].unique()[0]

In [20]:
# from st_science rename the score_other column to score so that it we can stack across all the topics and the belief and
# crtainty columns to not include the topic '_prefix' or the other '_suffix'
st_science.rename(columns={'science_other':'score', 'science_belief_other':'belief', 'science_certainty_other':'certainty'}, inplace=True)
st_math.rename(columns={'math_other':'score', 'math_belief_other':'belief', 'math_certainty_other':'certainty'}, inplace=True)
st_verbal.rename(columns={'verbal_other':'score', 'verbal_belief_other':'belief', 'verbal_certainty_other':'certainty'}, inplace=True)
st_pop.rename(columns={'pop_other':'score', 'pop_belief_other':'belief', 'pop_certainty_other':'certainty'}, inplace=True)
st_sports.rename(columns={'sports_other':'score', 'sports_belief_other':'belief', 'sports_certainty_other':'certainty'}, inplace=True)
st_us.rename(columns={'us_other':'score', 'us_belief_other':'belief', 'us_certainty_other':'certainty'}, inplace=True)


# stack all the topic tables into one larger data frame called st_updates
st_updates = pd.concat([st_science, st_math, st_verbal, st_pop, st_sports, st_us])
st_updates.reset_index(inplace=True, drop=True)

In [21]:
# add a colun that indicates the treatment
st_updates['treatment'] = 'stereotype'

# turn the type column into an integer
st_updates['type'] = st_updates['type'].astype(int)

# add a column that indicates if the participant was overconfident
st_updates['overconfident'] = 0
st_updates.loc[st_updates['type']<st_updates['belief'], 'overconfident'] = 1

# add a column that indicates if the participant was underconfident
st_updates['underconfident'] = 0
st_updates.loc[st_updates['type']>st_updates['belief'], 'underconfident'] = 1

# add a column that indicates if the participant was correct
st_updates['correct']=0
st_updates.loc[st_updates['type']==st_updates['belief'], 'correct'] = 1

# create a table for each pair of theta and omega (type and rate) the subindexes are in that order
#low types
updates_ll = st_updates.loc[(st_updates['type']==0) & (st_updates['rate']==0), :]
updates_lm = st_updates.loc[(st_updates['type']==0) & (st_updates['rate']==1), :]
updates_lh = st_updates.loc[(st_updates['type']==0) & (st_updates['rate']==2), :]

# mid types
updates_ml = st_updates.loc[(st_updates['type']==1) & (st_updates['rate']==0), :]
updates_mm = st_updates.loc[(st_updates['type']==1) & (st_updates['rate']==1), :]
updates_mh = st_updates.loc[(st_updates['type']==1) & (st_updates['rate']==2), :]

# high types
updates_hl = st_updates.loc[(st_updates['type']==2) & (st_updates['rate']==0), :]
updates_hm = st_updates.loc[(st_updates['type']==2) & (st_updates['rate']==1), :]
updates_hh = st_updates.loc[(st_updates['type']==2) & (st_updates['rate']==2), :]

In [22]:
st_updates.loc[st_updates['overconfident']==1, 'misspecification'] = 'over'
st_updates.loc[st_updates['underconfident']==1, 'misspecification'] = 'under'
st_updates.loc[st_updates['correct']==1, 'misspecification'] = 'correct'


In [23]:
# merge the data from both treatments into a table called updates
updates = pd.concat([ego_updates, st_updates])

In [24]:
characteristics_ego = ego_wide[['participant.code', 
                        'participant.gender',
                        'participant.nationality']]

characteristics_stereo = stereo_wide[['participant.code', 
                        'participant.gender',
                        'participant.nationality']]

characteristics = pd.concat([characteristics_ego, characteristics_stereo], axis=0, sort=False)


# rename the columns to remove the prefix 'participant.'
characteristics.columns = [col.replace('participant.','') for col in characteristics.columns]

In [25]:
# merge the data from characteristics with the data from updates on code. keep everything from updates
updates = updates.merge(characteristics, on='code', how='left')

### Maka a table to look into stereotypes

In [26]:
# get the gender and nationality of the other 
characteristics_other = stereo_signal_other_wide.groupby(['code', 'gender_other', 'nationality_other']).mean('round_number')

characteristics_other.reset_index(inplace=True)


# get the beliefs about themselves
beliefs_self_stereo = stereo_signal_other_wide[['us_belief_self', 'us_certainty_self',
                                                'math_belief_self', 'math_certainty_self',
                                                'pop_belief_self', 'pop_certainty_self',
                                                'verbal_belief_self', 'verbal_certainty_self',
                                                'science_belief_self', 'science_certainty_self',
                                                'sports_belief_self', 'sports_certainty_self', 'code', 'round_number']]


In [27]:
# transform table beliefs_self_stereo into a long format takeing the prefixes 'us_', 'math_', 'pop_', 'science_', 'verbal_' 
# and 'sports_' and make it a column called topic
beliefs_self_stereo_long = pd.melt(beliefs_self_stereo[beliefs_self_stereo['round_number']==1], id_vars=['code', 'round_number'],
                                      value_vars=['us_belief_self', 'us_certainty_self',
                                                'math_belief_self', 'math_certainty_self',
                                                'pop_belief_self', 'pop_certainty_self',
                                                'verbal_belief_self', 'verbal_certainty_self',
                                                'science_belief_self', 'science_certainty_self',
                                                'sports_belief_self', 'sports_certainty_self'],
                                        var_name='topic', value_name='value')

# split the topic column into two columns, one for the topic and one for the belief or certainty
beliefs_self_stereo_long[['topic','belief_or_certainty', 'whose']] = beliefs_self_stereo_long['topic'].str.split('_', expand=True)

# take only the rows in which the value is not null
beliefs_self_stereo_long = beliefs_self_stereo_long.loc[beliefs_self_stereo_long['value'].notnull(), :]

# split the table into two, one for the beliefs and one for the certainties
beliefs_self_stereo_long_belief = beliefs_self_stereo_long.loc[beliefs_self_stereo_long['belief_or_certainty']=='belief', :]
beliefs_self_stereo_long_certainty = beliefs_self_stereo_long.loc[beliefs_self_stereo_long['belief_or_certainty']=='certainty', :]

# rename the column value to belief
beliefs_self_stereo_long_belief.rename(columns={'value':'belief'}, inplace=True)
# rename the column value to certainty
beliefs_self_stereo_long_certainty.rename(columns={'value':'certainty'}, inplace=True)

# merge the two tables beliefs_self_stereo_long_belief and beliefs_self_stereo_long_certainty
beliefs_self_stereo_long = beliefs_self_stereo_long_belief.merge(beliefs_self_stereo_long_certainty, on=['code', 'round_number', 'topic'], how='left')

beliefs_self_stereo_long = beliefs_self_stereo_long[['code', 'belief', 'certainty', 'topic']]

own_belief_other_characteristics = beliefs_self_stereo_long.merge(characteristics_other, on='code')


/var/folders/fz/pp86wz7x4h514cs2lxtvtq3c0000gn/T/ipykernel_29368/3038939621.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beliefs_self_stereo_long_belief.rename(columns={'value':'belief'}, inplace=True)
/var/folders/fz/pp86wz7x4h514cs2lxtvtq3c0000gn/T/ipykernel_29368/3038939621.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beliefs_self_stereo_long_certainty.rename(columns={'value':'certainty'}, inplace=True)


In [28]:
own_belief_other_characteristics= own_belief_other_characteristics[['belief', 'certainty', 'topic', 'gender_other', 'nationality_other', 'code']]

In [29]:
# rename the topics to match those in updates
own_belief_other_characteristics.loc[own_belief_other_characteristics['topic']=='us', 'topic'] = 'US Geography'
own_belief_other_characteristics.loc[own_belief_other_characteristics['topic']=='math', 'topic'] = 'Math'
own_belief_other_characteristics.loc[own_belief_other_characteristics['topic']=='pop', 'topic'] = 'Pop-Culture and Art'
own_belief_other_characteristics.loc[own_belief_other_characteristics['topic']=='science', 'topic'] = 'Science and Technology'
own_belief_other_characteristics.loc[own_belief_other_characteristics['topic']=='verbal', 'topic'] = 'Verbal'
own_belief_other_characteristics.loc[own_belief_other_characteristics['topic']=='sports', 'topic'] = 'Sports and Video Games'


In [30]:
beliefs_other = updates.loc[updates['round_number']==1, ['belief', 'topic', 'code', 'certainty']]

In [31]:
len(beliefs_other['code'].unique())

86

In [32]:
own_belief_other_characteristics

,belief,certainty,topic,gender_other,nationality_other,code
0,0,75,US Geography,Female,US National,0kxsrg73
1,2,75,Math,Female,US National,0kxsrg73
2,1,75,Pop-Culture and Art,Female,US National,0kxsrg73
3,0,75,Verbal,Female,US National,0kxsrg73
4,1,50,Science and Technology,Female,US National,0kxsrg73
...,...,...,...,...,...,...
241,1,75,Math,Male,non-US National,zwzznpo6
242,0,75,Pop-Culture and Art,Male,non-US National,zwzznpo6
243,0,75,Verbal,Male,non-US National,zwzznpo6
244,1,75,Science and Technology,Male,non-US National,zwzznpo6


In [33]:
own_belief_other_characteristics.rename(columns={'belief':'belief_self', 'certainty':'certainty_self'}, inplace=True)

# merge with the beliefs and certainty of the other and the self characteristics
own_belief_other_characteristics = own_belief_other_characteristics.merge(beliefs_other, how='left')

In [34]:
len(own_belief_other_characteristics)

246

In [35]:
own_characteristics_other_beliefs = updates.loc[(updates['treatment']=='stereotype') & (updates['round_number']==1), ['gender', 'nationality', 'code', 'belief', 'certainty', 'misspecification', 'topic', 'round_number']]
own_characteristics_other_beliefs.rename(columns={'belief':'belief_other', 'certainty':'certainty_other'}, inplace=True)

stereo_characteristics_beliefs = own_belief_other_characteristics.merge(own_characteristics_other_beliefs, on=['code', 'topic'], how='left')

In [36]:
own_characteristics_other_beliefs


,gender,nationality,code,belief_other,certainty_other,misspecification,topic,round_number
2970,Female,non-US National,0kxsrg73,1,50,correct,Science and Technology,1
2981,Female,non-US National,16av43c2,2,100,correct,Science and Technology,1
2992,Female,US National,1zb27tea,0,33,under,Science and Technology,1
3003,Female,non-US National,36b7nxpg,0,50,under,Science and Technology,1
3014,Male,non-US National,3cv0gsso,1,33,correct,Science and Technology,1
...,...,...,...,...,...,...,...,...
5621,Male,non-US National,uz5m20q1,1,33,correct,US Geography,1
5632,Female,non-US National,v3yunf9n,1,33,correct,US Geography,1
5643,Male,non-US National,vzs176mt,0,100,under,US Geography,1
5654,Female,non-US National,wursesku,2,75,over,US Geography,1


In [37]:
stereo_characteristics_beliefs

,belief_self,certainty_self,topic,gender_other,nationality_other,code,belief,certainty,gender,nationality,belief_other,certainty_other,misspecification,round_number
0,0,75,US Geography,Female,US National,0kxsrg73,1,50,Female,non-US National,1,50,correct,1
1,2,75,Math,Female,US National,0kxsrg73,2,75,Female,non-US National,2,75,over,1
2,1,75,Pop-Culture and Art,Female,US National,0kxsrg73,1,75,Female,non-US National,1,75,correct,1
3,0,75,Verbal,Female,US National,0kxsrg73,1,75,Female,non-US National,1,75,over,1
4,1,50,Science and Technology,Female,US National,0kxsrg73,1,50,Female,non-US National,1,50,correct,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,1,75,Math,Male,non-US National,zwzznpo6,1,33,Male,non-US National,1,33,correct,1
242,0,75,Pop-Culture and Art,Male,non-US National,zwzznpo6,0,33,Male,non-US National,0,33,under,1
243,0,75,Verbal,Male,non-US National,zwzznpo6,0,50,Male,non-US National,0,50,under,1
244,1,75,Science and Technology,Male,non-US National,zwzznpo6,1,33,Male,non-US National,1,33,correct,1


In [38]:
# check the data types of the columns
updates.dtypes

# set the columns effort, fails, last_button, score, signal, belief, certainty to be integers
updates[['effort', 'fails', 'last_button', 'score', 'signal', 'belief', 'certainty']] = updates[['effort', 'fails', 'last_button', 'score', 'signal', 'belief', 'certainty']].astype(int)


In [39]:
# add a column that turns the misspecification column into a numeric variable. -1 if under, 0 if correct and 1 if over
updates['misspecification_num'] = 0
updates.loc[updates['misspecification']=='over', 'misspecification_num'] = 1
updates.loc[updates['misspecification']=='under', 'misspecification_num'] = -1

# save data into a csv file
updates.to_csv('../Clean/updates.csv', index=False)
ego_updates.to_csv('../Clean/ego_updates.csv', index=False)
st_updates.to_csv('../Clean/stereo_updates.csv', index=False)
stereo_characteristics_beliefs.to_csv('../Clean/stereo_characteristics_beliefs.csv', index=False)

In [40]:
stereo_characteristics_beliefs.columns

Index(['belief_self', 'certainty_self', 'topic', 'gender_other',
       'nationality_other', 'code', 'belief', 'certainty', 'gender',
       'nationality', 'belief_other', 'certainty_other', 'misspecification',
       'round_number'],
      dtype='object')

In [41]:
print(str(len(updates['code'].unique())) + ' participants')
print (str(len(updates[updates['treatment']=='ego']['code'].unique())) + ' in the ego treatment')
print(str(len(updates[updates['treatment']=='stereotype']['code'].unique())) + ' in the stereotype treatment')

86 participants
45 in the ego treatment
41 in the stereotype treatment


# The Variables in the Clean tables

### Updates table
- **code**: unique subject level id
- **round_number**: the round number
- **effort** - their choice saved as 0, 1, or 2
- **fails** - number of fails that they got in their sample of 10 binary gambles
- **last_button** - the last button that they clicked to reveal a matrix. Proxy for their belief about theta in each period (0, 1, 2)
- **score** - their true score in the quiz ranges from 0 to 20
- **topic** - the topic that they were answering questions about
- **signal** - the number of successes that they got in the sample of 10 gambles
- **belief** - their belief about their score in the quiz (reported after they took all quizzes it is 0 (beleived score 0 to 5), 1 (believed score 6 tp 15), 2 (believed score 16 or more))
- **certainty** - their certainty about their score in the quiz. random guess (33), other equally likely (50), somewhat certain (75) and completely certain (100)
- **type** - their true type (0, 1, 2)
- **rate** - the exogenour parameter
- **treatment** - ego or stereotype
- **overconfident** - indicator for when the reported belief was higher than the true score
- **underconfident** - indicator for when the reported belief was lower than the true score
- **correct** - indicator for when the reported belief was equal to the true score
- **misspecification** - under, over or correct (at the beginning of the task)
- **gender** - own gender
- **nationality** - own nationality
- **misspecification_num** - -1 if under, 0 if correct, 1 if over 

